<a href="https://colab.research.google.com/github/sarelWeinberger/data_for_lm/blob/master/AWS_CDK_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers -q

import torch
from transformers import CodeGenForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('CarperAI/diff-codegen-6b-v2')
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CodeGenForCausalLM.from_pretrained('CarperAI/diff-codegen-6b-v2').to(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.7 MB/s eta 0:00:00


KeyboardInterrupt: ignored

 
 
`<MSG>` Testing Cases:

*   EC2 instances with public IP addresses shouldn't allow for unrestricted traffic to their subnets
*   EC2 instance has a sensitive port connection exposed to the entire network
* ECS Service's security group should not allow unrestricted access to all ports from all IPv4 addresses




In [ ]:
prompt = '''
<BEF> import aws_cdk.aws_ec2 as ec2


vpc = ec2.Vpc(
    self,
    "Vpc",
    cidr="10.0.0.0/16"
)

vpc_connector = apprunner.VpcConnector(
    self,
    "VpcConnector",
    vpc=vpc,
    vpc_subnets=vpc.select_subnets(subnet_type=ec2.SubnetType.PUBLIC),
    vpc_connector_name="MyVpcConnector"
)
sg_ec2_instance = aws_ec2.SecurityGroup(
    self,
    'EC2InstanceSG',
    vpc=vpc,
    allow_all_outbound=True, 
    description='security group for EC2 Instance',
    security_group_name=EC2_SG_NAME
)

### IT SHOULD to REQUIRE THIS:
sg_ec2_instance.add_ingress_rule(
 peer=aws_ec2.Peer.ipv4("0.0.0.0/0"),
 connection=aws_ec2.Port.tcp(22)
)

<MSG> EC2 instance has a sensitive port connection exposed to the entire network
'''


inputs = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
model.config.use_cache = True
model_output = model.generate(**inputs, temperature=0.8, max_length=600)
for i in range(model_output.size(0)):
    print(tokenizer.decode(model_output[i]))
    print('\n' + '-'*10 + '\n')  # Separating long generated texts

`<MSG>` Testing Cases:

*   The HTTP port is open in a Security Group
*   No security group should allow unrestricted egress access
* The IP address in a DB Security Group must not be '0.0.0.0/0' (IPv4) or '::/0' (IPv6).



In [ ]:
prompt = '''
<BEF> sg_ec2_instance = aws_ec2.SecurityGroup(
    self,
    'EC2InstanceSG',
    allow_all_outbound=True, 
    description='security group for EC2 Instance',
)
sg_ec2_instance.add_ingress_rule(
  peer=aws_ec2.Peer.ipv4("0.0.0.0/0"),
  connection=aws_ec2.Port.tcp(80)
)

<MSG> The HTTP port is open in a Security Group
'''


inputs = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
model.config.use_cache = True
model_output = model.generate(**inputs, temperature=0.8, max_length=300)
for i in range(model_output.size(0)):
    print(tokenizer.decode(model_output[i]))
    print('\n' + '-'*10 + '\n')  # Separating long generated texts

`<MSG>` Testing Cases:

*  SageMaker Notebook must be placed in a VPC
* KmsKeyId attribute should be defined
* SageMaker must have disabled internet access and root access for Creating Notebook Instances.
* SageMaker Data Encryption Disabled



In [ ]:
prompt = '''
<BEF> sg_sagemaker_notebook_instance = aws_ec2.SecurityGroup(self, "SageMakerNotebookSG",
      ### vpc=vpc, 
      allow_all_outbound=True,
      description='Security group with no ingress rule',
      security_group_name='sagemaker-nb-{}-sg'.format(''.join(random.sample((string.ascii_letters), k=5)))
    )
    sg_sagemaker_notebook_instance.add_ingress_rule(peer=sg_sagemaker_notebook_instance, connection=aws_ec2.Port.all_traffic(), description='sagemaker notebook security group')
   
<MSG> SageMaker Notebook must be placed in a VPC
'''


inputs = tokenizer(prompt, return_tensors='pt', padding=True).to(device)
model.config.use_cache = True
model_output = model.generate(**inputs, temperature=0.8, max_length=300)
for i in range(model_output.size(0)):
    print(tokenizer.decode(model_output[i]))
    print('\n' + '-'*10 + '\n')  # Separating long generated texts